In [1]:
import logging
import traceback

import panel as pn

pn.extension('terminal', console_output='disable')
#pn.extension('terminal')

In [2]:
logger = logging.getLogger('panel.Debugger')

In [3]:
debug = pn.widgets.Debugger(
    name='Debugger (level=DEBUG)',
    only_last=False,
    # at logging.DEBUG level there are many messages from Panel,
    # so to avoid flooding the Debugger widget, limit it to application logger
    level=logging.DEBUG, logger_names=['panel.Debugger'])
debug

Debugger(button_css_classes=['debugger-card-button'], collapsed=True, css_classes=['debugger-card'], header_css_classes=['debugger-card-header'], level=10, logger_names=['panel.Debugger'], only_last=False, title_css_classes=['debugger-card-title'])
    [0] Row(align=('end', 'start'), sizing_mode='stretch_width')
        [0] Markdown(str)
        [1] HSpacer()
        [2] DebuggerButtons(stylesheets=['https://cdn.holoviz.org/...])
    [1] Terminal(id=140107491269504)

In [4]:
logger.debug("DEBUG")
logger.info("INFO")
logger.warning("WARNING")
logger.error("ERROR")

2024-10-24 12:27:30,573 WARNING: panel.Debugger - WARNING
2024-10-24 12:27:30,576 ERROR: panel.Debugger - ERROR


In [5]:
widget = pn.widgets.Select(name='Widget', options=['Biology', 'Chemistry', 'Physics'])
select = pn.widgets.Select(name='Select', options=['foo', 'bar', 'baz'])

In [6]:
#@pn.cache
def markdown_contents(value: str) -> str:
    #bt = traceback.format_stack()
    #logger.debug(f"markdown_contents({value=})\n{"\n".join(bt)}")
    logger.debug(f"markdown_contents({value=})")
    return f"""Value is _{value}_"""

In [7]:
#@pn.cache
def append_exclamation(value: str) -> str:
    logger.debug(f"append_exclamation({value=})")
    return value + "!!!"

In [8]:
logger.info("creating bindings...")

In [9]:
bound_markdown_contents = pn.bind(
    markdown_contents, widget,
    #watch=True,
)

In [10]:
bound_append_exclamation = pn.bind(
    append_exclamation, bound_markdown_contents,
)
logger.info("showing widget...")
pn.Column(widget, pn.pane.Str(bound_append_exclamation))

Column
    [0] Select(name='Widget', options=['Biology', 'Chemistry', ...], value='Biology')
    [1] Str(str)

In [11]:
logger.info("creating reactive components...")

In [12]:
markdown_contents_rx = pn.rx(markdown_contents)(select)
append_exclamation_rx = pn.rx(append_exclamation)(markdown_contents_rx)

In [13]:
pn.Column(select, pn.pane.Str(append_exclamation_rx))

Column
    [0] Select(options=['foo', 'bar', 'baz'], value='foo')
    [1] Str(str)

In [14]:
logger.info("creating template...")

In [15]:
template = pn.template.MaterialTemplate(
    site="debugger",
    title="Debugger demo",
    sidebar=[
        widget,
        select,
    ],
    main=[
        pn.pane.Markdown(bound_append_exclamation),
        pn.pane.Markdown(append_exclamation_rx),
    ],
)

In [16]:
template.main.extend([
    pn.layout.Divider(),
    pn.Card(debug, header="Debugger"),
])

In [17]:
template.servable(); # The ; is needed in the notebook to not display the template. Its not needed in a script